# Clark County Only, Injury Type as Target

## Imports and Setup

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import psycopg2 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Load in password
from secret import secret, database, username, host

# Establish connection to database
conn = psycopg2.connect(
    host = host,
    database = database,
    user =username,
    password = secret
)

In [3]:
# Load the data
query = 'SELECT * FROM clark_co_traffic'
traffic_df = pd.read_sql_query(query, conn)

/var/folders/f3/5zcd9_1j7d3546ws7g9t5fw00000gp/T/ipykernel_6132/3027837082.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  traffic_df = pd.read_sql_query(query, conn)


## Preprocess Data

In [4]:
# Filter for Clark County
traffic_df = traffic_df.loc[traffic_df['county'] == 'CLARK']
traffic_df.drop(columns = ['county'], inplace = True)

In [5]:
# Drop unnecessary columns
traffic_df.drop(columns = ['objectid','crash_severity','accident_rec_num','primary_street', 'secondary_street', 'fatalities', 'injured' ], inplace = True)

In [6]:
# Move PDO values from property_damage_only column info to injury_type column
traffic_df['injury_type'] = np.where(traffic_df['property_damage_only'] == 'PDO', traffic_df['property_damage_only'], traffic_df['injury_type'])

In [7]:
# Drop property damage only column
traffic_df.drop(columns = ['property_damage_only'], inplace = True)

In [8]:
# Split single column with multiple data points into separate columns
traffic_df[['v1_driver_factors_1', 'v1_driver_factors_2','v1_driver_factors_3', 'v1_driver_factors_4']] = traffic_df['v1_driver_factors'].str.split(':', expand=True)

traffic_df[['v1_vehicle_factors_1', 'v1_vehicle_factors_2','v1_vehicle_factors_3', 'v1_vehicle_factors_4', 'v1_vehicle_factors_5']] = traffic_df['v1_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v1_all_events_1', 'v1_all_events_2','v1_all_events_3', 'v1_all_events_4', 'v1_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['v2_driver_factors_1', 'v2_driver_factors_2']] = traffic_df['v2_driver_factors'].str.split(':', expand=True)

traffic_df[['v2_vehicle_factors_1', 'v2_vehicle_factors_2','v2_vehicle_factors_3', 'v2_vehicle_factors_4', 'v2_vehicle_factors_5']] = traffic_df['v2_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v2_all_events_1', 'v2_all_events_2','v2_all_events_3', 'v2_all_events_4', 'v2_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['nonmotorist_factors_1', 'nonmotorist_factors_2','nonmotorist_factors_3', 'nonmotorist_factors_4', 'nonmotorist_factors_5']] = traffic_df['nonmotorist_factors'].str.split(':', expand=True)

# traffic_df[['factors_roadway_1', 'factors_roadway_2','factors_roadways_3']] = traffic_df['factors_roadway'].str.split(':', expand=True)

traffic_df[['hwy_factors_1', 'hwy_factors_2','hwy_factors_3', 'hwy_factors_4']] = traffic_df['hwy_factors'].str.split(':', expand=True)

traffic_df.drop(columns = ['v1_driver_factors','v1_vehicle_factors','v1_all_events','v2_driver_factors','v2_vehicle_factors', 'v2_all_events', 'nonmotorist_factors', 'hwy_factors',], inplace = True)

In [9]:
traffic_df['injury_type'].value_counts()

PDO    91631
C      57266
B      17300
A       2707
K        788
U        128
Name: injury_type, dtype: int64

In [10]:
# Change target values to numeric
# Values are based on Nevada Traffic Records Coordinating Committee Data Dictionary,
# https://zerofatalitiesnv.com/app/uploads/2021/04/2021-01-NV-TRCC-Data-Dictionary.pdf
injury_num = {
    'K': 5,
    'A': 4,
    'B': 3,
    'C': 2,
    'PDO': 1,
    'U': 0,      
}

traffic_df['injury_type'] = traffic_df['injury_type'].apply(lambda x: injury_num[x])

traffic_df['injury_type'].value_counts()

2    91631
3    57266
4    17300
5     2707
6      788
1      128
Name: injury_type, dtype: int64

In [11]:
# Extract datetime values into separate columns
traffic_df['crash_month'] = pd.DatetimeIndex(traffic_df['crash_date']).month
traffic_df['crash_day'] = pd.DatetimeIndex(traffic_df['crash_date']).day
traffic_df['crash_day_of_week'] = pd.DatetimeIndex(traffic_df['crash_date']).dayofweek
traffic_df['crash_hour'] = pd.DatetimeIndex(traffic_df['crash_time']).hour


In [13]:
# Drop original datetime columns
traffic_df.drop(columns = ['crash_date', 'crash_time'], inplace = True)

In [14]:
traffic_df.dtypes

x                    float64
y                    float64
crash_year             int64
weather               object
injury_type            int64
                      ...   
hwy_factors_4         object
crash_month            int64
crash_day              int64
crash_day_of_week      int64
crash_hour             int64
Length: 62, dtype: object

In [15]:
# Get names of all columns with dtypes of "object"
sel_cols = list(traffic_df.select_dtypes(include='object'))


In [16]:
# Preprocessing to convert all strings to numeric values
traffic_df_encoded = pd.get_dummies(traffic_df, columns = sel_cols)
traffic_df_encoded.head()

,x,y,crash_year,injury_type,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_3_ OTHER ENVIRONMENTAL,hwy_factors_3_ OTHER HIGHWAY,hwy_factors_3_ ROAD OBSTRUCTION,"hwy_factors_3_ RUTS, HOLES, BUMPS",hwy_factors_3_ SHOULDERS,hwy_factors_3_ VISUAL OBSTRUCTION(S),hwy_factors_3_ WEATHER,"hwy_factors_3_ WET, ICY, SNOW, SLUSH",hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY),"hwy_factors_4_ WET, ICY, SNOW, SLUSH"
0,-115.106709,36.236043,2016,2,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,2016,2,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,2016,2,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0
5,-115.278964,36.136313,2016,6,1,35,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0
6,-115.162277,36.232014,2016,3,1,43,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0


## Model for Clark County with Injury Type as Target and All Other Columns as Features

In [17]:
# Separate data into features and target
y = traffic_df_encoded['injury_type'].values
X = traffic_df_encoded.drop(columns = 'injury_type')

In [18]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)
X_train.shape

(127365, 1231)

In [19]:
# Scale the model
scaler = StandardScaler() 

# Fit the scaler with the training data
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Create and fit the Random Forest model
rf_model = RandomForestClassifier(n_estimators = 128)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [21]:
# Create predictions
predictions = rf_model.predict(X_test_scaled)
results = pd.DataFrame({
    "Prediction": predictions,
    "Actual": y_test
}).reset_index(drop = True)
results.head()

,Prediction,Actual
0,2,2
1,3,3
2,3,2
3,2,3
4,3,3


In [22]:
# Assess accuracy score
accuracy = accuracy_score(y_test, predictions)
accuracy

0.6460958662112826

In [23]:
# Generate confusion matrix and dataframe
cm = confusion_matrix(y_test, predictions)

index_values = ['Actual: Unknown', 
                'Actual: Property Damage Only', 
                'Actual: Possible Injury', 
                'Actual: Suspected Minor Injury', 
                'Actual: Suspected Serious Injury', 
                'Actual: Fatal Injury']

column_values = ['Predicted: Unknown', 
                'Predicted: Property Damage Only', 
                'Predicted: Possible Injury', 
                'Predicted: Suspected Minor Injury', 
                'Predicted: Suspected Serious Injury', 
                'Predicted: Fatal Injury']

cm_df = pd.DataFrame(cm, index = index_values, columns = column_values)
cm_df
                  

,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,13,8,11,0,0
Actual: Property Damage Only,0,18501,4328,77,2,0
Actual: Possible Injury,0,5484,8468,351,11,2
Actual: Suspected Minor Injury,0,1306,2582,414,20,3
Actual: Suspected Serious Injury,0,169,305,169,27,7
Actual: Fatal Injury,0,65,43,59,10,20


## Results for Clark County with Injury Type as Target and All Other Columns as Features

In [24]:
# Display results
print('Confusion Matrix')
display(cm_df)
print(f'\nAccuracy Score: {accuracy}\n')
print('Classification Report')
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,13,8,11,0,0
Actual: Property Damage Only,0,18501,4328,77,2,0
Actual: Possible Injury,0,5484,8468,351,11,2
Actual: Suspected Minor Injury,0,1306,2582,414,20,3
Actual: Suspected Serious Injury,0,169,305,169,27,7
Actual: Fatal Injury,0,65,43,59,10,20



Accuracy Score: 0.6460958662112826

Classification Report
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        32
           2       0.72      0.81      0.76     22908
           3       0.54      0.59      0.56     14316
           4       0.38      0.10      0.15      4325
           5       0.39      0.04      0.07       677
           6       0.62      0.10      0.17       197

    accuracy                           0.65     42455
   macro avg       0.44      0.27      0.29     42455
weighted avg       0.62      0.65      0.62     42455



/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Importances for Clark County with Injury Type as Target and All Other Columns as Features

In [25]:
# Calculate importance of features
importances = rf_model.feature_importances_
important = sorted(zip(importances, X.columns), reverse = True)
important

[(0.05457054485297779, 'y'),
 (0.05450121327270793, 'v1_driver_age'),
 (0.05401595354301524, 'x'),
 (0.04795770780992282, 'v2_driver_age'),
 (0.046772685196051454, 'crash_day'),
 (0.0435151128106328, 'crash_hour'),
 (0.039984688468446224, 'crash_month'),
 (0.03534417703322784, 'crash_day_of_week'),
 (0.02302574129124955, 'crash_year'),
 (0.02284611815144245, 'factors_roadway_UNKNOWN'),
 (0.019163240402188253, 'lighting_UNKNOWN'),
 (0.01774361999522802, 'factors_roadway_DRY'),
 (0.013957344740033913, 'hwy_factors_1_UNKNOWN'),
 (0.01324555690394835, 'hwy_factors_1_NONE'),
 (0.013202447105025474, 'lighting_DAYLIGHT'),
 (0.012621915961743438, 'total_vehicles'),
 (0.012068326621819512, 'v1_type_SEDAN, 4 DOOR'),
 (0.00915164593678924, 'v2_type_SEDAN, 4 DOOR'),
 (0.008997451313453671, 'v1_type_CARRY-ALL'),
 (0.008865194720779254, 'v1_vehicle_factors_1_FAILED TO YIELD RIGHT OF WAY'),
 (0.008590071907621592, 'v1_action_GOING STRAIGHT'),
 (0.00821069859834884, 'v1_driver_factors_1_APPARENTLY NOR